In [1]:
import numpy as np
import matplotlib.pyplot as plt




NameError: name 'a' is not defined

In [16]:
'''
cleandata=np.random.normal(0,0.1,1000)
noisedata=np.random.uniform(0,0.1,1000)
mixdata=cleandata+noisedata

noisemean=np.mean(noisedata)
noisestd=np.std(noisedata)

#plt.figure()
#plt.plot(b,'b-o')

cleancount, cleanbins, cleanignored=plt.hist(cleandata,25,normed=True)
plt.plot(cleanbins,1/(np.sqrt(2*np.pi*std**2))*
         np.exp(-(cleanbins - mean)**2/(2 * std**2)),
       linewidth=2,color='r')

plt.figure()
mixcount, mixbins, mixignored=plt.hist(mixdata,25,normed=True,color='g')
plt.plot(mixbins,1/(np.sqrt(2*np.pi*std**2))*
         np.exp(-(mixbins - mean)**2/(2 * std**2)),
       linewidth=2,color='r')

'''

"\ncleandata=np.random.normal(0,0.1,1000)\nnoisedata=np.random.uniform(0,0.1,1000)\nmixdata=cleandata+noisedata\n\nnoisemean=np.mean(noisedata)\nnoisestd=np.std(noisedata)\n\n#plt.figure()\n#plt.plot(b,'b-o')\n\ncleancount, cleanbins, cleanignored=plt.hist(cleandata,25,normed=True)\nplt.plot(cleanbins,1/(np.sqrt(2*np.pi*std**2))*\n         np.exp(-(cleanbins - mean)**2/(2 * std**2)),\n       linewidth=2,color='r')\n\nplt.figure()\nmixcount, mixbins, mixignored=plt.hist(mixdata,25,normed=True,color='g')\nplt.plot(mixbins,1/(np.sqrt(2*np.pi*std**2))*\n         np.exp(-(mixbins - mean)**2/(2 * std**2)),\n       linewidth=2,color='r')\n\n"

In [20]:
import numpy as np
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


from simdata import get_mldata



######################################
# Getting data from data sources
######################################

#=========================
#get data from csv files
#=========================

mixdata_sim_filedir='../resources/gamedata/mixdata_sim.csv'
inerdata_sim_filedir='../resources/gamedata/interdata_sim.csv'
biodata_sim_filedir='../resources/gamedata/biodata_sim.csv'

mixdata_sim=get_mldata(mixdata_sim_filedir)
interdata_sim=get_mldata(inerdata_sim_filedir)
biodata_sim=get_mldata(biodata_sim_filedir)


mixdata_labels=[]
mixdata_features=[]
for i in mixdata_sim:
    mixdata_labels=mixdata_labels+[i[14]]
    mixdata_features=mixdata_features+[i[0:14]]


mixdata_sim [7.0, 3.0, 5.0, 5.0, 65.0, 1.0, 0.7, 0.1, 187.0, 16.0, 18.0, 6.8, 3.3, 0.9, 1.0]
interdata [7.0, 3.0, 5.0, 5.0, 1.0]
biodata [65.0, 1.0, 0.7, 0.1, 187.0, 16.0, 18.0, 6.8, 3.3, 0.9, 1.0]


In [14]:
############################################
# Clustering based on interdata
############################################

#===========================
# get data for clustering
#===========================

features_km=[]

for i in mixdata_sim:
    features_km=features_km+[i[0:4]]

features_km_np=np.array(features_km)

features_km_np = (features_km_np - features_km_np.mean(axis=0)) / features_km_np.std(axis=0)

#=================================
# cluster interdata by kmeans
#=================================

clu_km=KMeans(n_clusters=2,random_state=0).fit(features_km_np)

#labels_pre_km=kmeans.predict(features_interdata_sim[400:579])
#testlabels_true=label_interdata_sim[400:579]
#metrics.adjusted_rand_score(testlabels_true,testlabels_pre_km)

#============================
#labels for biodata
#============================
labels_pre_km=clu_km.labels_

labels_pre_km_centers=clu_km.cluster_centers_

#######################################
# Classify samples
#######################################

#================================
# data for classifying analysis
#================================
labels_svm=[]

for i in labels_pre_km:
    if i==0:
        #labels_svm=labels_svm+[labels_pre_km_centers[0]]
        labels_svm=labels_svm+[[0.]]
    else:
        #labels_svm=labels_svm+[labels_pre_km_centers[1]]
        labels_svm=labels_svm+[[1.]]

labels_svm_np = np.array(labels_svm)

features_svm=[]
for i in mixdata_sim:
    features_svm=features_svm+[i[4:14]]

features_svm_np = np.array(features_svm)
features_svm_np = (features_svm_np - features_svm_np.mean(axis=0)) / features_svm_np.std(axis=0)# follow natural distribution


train_features, test_features, train_labels, test_labels = train_test_split(features_svm_np, labels_svm_np, test_size=0.33)



#============================
#classify biodata by SVM
#============================

clf_svm=svm.SVC(C=1)
clf_svm.fit(train_features,train_labels)
labels_pre_svm=clf_svm.predict(test_features)


#============================
# Evaluation
#============================

acc_svm=accuracy_score(test_labels,labels_pre_svm)
print("acc_svm:",acc_svm)

#=================
# analysis
#=================
biolabels=[]
interlabels=[]
for i in range(len(biodata_sim)):
    biolabels=biolabels+[biodata_sim[i][10]]
biolabels_np=np.array(biolabels)

cluster_labels=[]
for i in range(len(labels_pre_km)):
    if labels_pre_km[i]==1:
        cluster_labels=cluster_labels+[0]
    else:
        cluster_labels = cluster_labels + [1]

acc_clu_bio=accuracy_score(biolabels,cluster_labels)
print("acc_clu_bio:",acc_clu_bio)




acc_svm: 0.734375
acc_clu_bio: 1.0


/Users/cancui/workspace/anaconda3/envs/abiba_analysis/lib/python3.5/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
#create noise
#
a=[1,2]

if 1 in a:
    print("yes")